# Trabajo de Consultas
Presentado por Carlos Dubón [@AmadeusCelta]

## Preparación del ámbiente

In [1]:
# Librerías a Utilizar
import pandas as pd
from datetime import timedelta

## Preparación de los datos

In [2]:
# Cargar las tablas
credit_record = pd.read_csv("credit_record.csv")  # Contiene Id, StatusDate, Status, DepositBalance
loan_applications = pd.read_csv("loan_applications.csv")  # Contiene Id, Application_Date

In [3]:
# Renombrar la primera columna (actualente sin nombre) como 'Correlative'
credit_record.rename(columns={credit_record.columns[0]: 'Correlative'}, inplace=True)
loan_applications.rename(columns={loan_applications.columns[0]: 'Correlative'}, inplace=True)

In [4]:
# Convertir las fechas a datetime
credit_record['Status_date'] = pd.to_datetime(credit_record['Status_date'])
loan_applications['application_date'] = pd.to_datetime(loan_applications['application_date'])

In [5]:
credit_record

,Correlative,ID,Status_date,STATUS,saldo_depositos
0,0,5008804,2023-01-26,C,516.19
1,1,5008804,2022-12-26,C,13190.50
2,2,5008804,2022-11-26,C,204.67
3,3,5008804,2022-10-26,C,1214.29
4,4,5008804,2022-09-26,C,335.20
...,...,...,...,...,...
777710,777710,5150487,2021-06-14,C,29653.00
777711,777711,5150487,2021-05-15,C,292.47
777712,777712,5150487,2021-04-14,C,717.48
777713,777713,5150487,2021-03-15,C,5631.45


In [6]:
loan_applications

,Correlative,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,application_date
0,0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,2023-11-30
1,1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,2023-06-06
2,2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,2021-10-20
3,3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,2023-12-20
4,4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,2023-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438458,438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0,2022-11-03
438459,438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0,2023-01-24
438460,438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0,2022-12-22
438461,438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0,2022-03-21


## Procesamiento de la información

In [7]:
# Paso 1: Obtener estados al cierre de cada mes
credit_record['YearMonth'] = credit_record['Status_date'].dt.to_period('M')  # Crear agrupación Año-Mes
credit_record['Row'] = credit_record.groupby(['ID', 'YearMonth'])['Status_date'].rank(ascending=False, method='first')  # Equivalente de ROW_NUMBER
record_status = credit_record[credit_record['Row'] == 1][['ID', 'Status_date', 'STATUS']]  # Filtrar última fecha del mes

record_status

,ID,Status_date,STATUS
0,5008804,2023-01-26,C
1,5008804,2022-12-26,C
2,5008804,2022-11-26,C
3,5008804,2022-10-26,C
4,5008804,2022-09-26,C
...,...,...,...
777710,5150487,2021-06-14,C
777711,5150487,2021-05-15,C
777712,5150487,2021-04-14,C
777713,5150487,2021-03-15,C


In [8]:
# Paso 2: Obtener el estado según la fecha de aplicación (-3 meses)
record_status['EOMONTH_StatusDate'] = record_status['Status_date'].dt.to_period('M').dt.to_timestamp('M')  # Fin del mes
loan_applications['EOMONTH_ApplicationDate'] = (loan_applications['application_date'] - pd.offsets.MonthEnd(3)).dt.to_period('M').dt.to_timestamp('M')  # Fin de mes -3
customer_status = record_status.merge(
    loan_applications[['ID', 'EOMONTH_ApplicationDate']], 
    left_on=['ID', 'EOMONTH_StatusDate'], 
    right_on=['ID', 'EOMONTH_ApplicationDate'], 
    how='inner'
)[['ID', 'Status_date', 'STATUS']]  # Relación entre STATUS y fecha de 

customer_status

,ID,Status_date,STATUS
0,5008805,2023-03-05,C
1,5008806,2021-07-04,X
2,5008812,2023-04-25,X
3,5008814,2022-12-29,0
4,5008821,2022-02-10,X
...,...,...,...
13181,5150475,2022-07-29,C
13182,5150477,2023-04-18,0
13183,5150478,2021-11-27,C
13184,5150481,2022-02-26,X


In [9]:
# Paso 3: Obtener registros para promedios (últimos 6 meses)
loan_applications['START_DATE'] = loan_applications['application_date'] - pd.DateOffset(months=6)  # Calcular fecha inicial
record_balance = credit_record.merge(
    loan_applications[['ID', 'application_date', 'START_DATE']], 
    on='ID', 
    how='inner'
)
record_balance_filtered = record_balance[
    (record_balance['Status_date'] >= record_balance['START_DATE']) & 
    (record_balance['Status_date'] < record_balance['application_date'])
]  # Filtrar rango de 6 meses

record_balance_filtered

,Correlative,ID,Status_date,STATUS,saldo_depositos,YearMonth,Row,application_date,START_DATE
21,21,5008805,2023-06-05,C,726.88,2023-06,1.0,2023-06-06,2022-12-06
22,22,5008805,2023-05-05,C,1180.00,2023-05,1.0,2023-06-06,2022-12-06
23,23,5008805,2023-04-05,C,891.26,2023-04,1.0,2023-06-06,2022-12-06
24,24,5008805,2023-03-05,C,442.38,2023-03,1.0,2023-06-06,2022-12-06
25,25,5008805,2023-02-03,C,2622.67,2023-02,1.0,2023-06-06,2022-12-06
...,...,...,...,...,...,...,...,...,...
777702,777702,5150487,2022-02-13,C,1017.75,2022-02,1.0,2022-03-20,2021-09-20
777703,777703,5150487,2022-01-13,C,279.59,2022-01,1.0,2022-03-20,2021-09-20
777704,777704,5150487,2021-12-14,C,183.83,2021-12,1.0,2022-03-20,2021-09-20
777705,777705,5150487,2021-11-13,C,1669.43,2021-11,1.0,2022-03-20,2021-09-20


In [10]:
# Paso 4: Calcular el promedio de los saldos
customer_balance = record_balance_filtered.groupby('ID')['saldo_depositos'].mean().round(2).reset_index()
customer_balance.rename(columns={'saldo_depositos': 'AVGDepositBalance'}, inplace=True)
customer_balance

,ID,AVGDepositBalance
0,5008805,3769.74
1,5008806,1929.04
2,5008811,1567.59
3,5008812,15259.63
4,5008814,872.08
...,...,...
16757,5150475,544.40
16758,5150477,2493.24
16759,5150478,2197.64
16760,5150481,778.93


In [11]:
# Paso 5: Integrar resultados finales
final_result = loan_applications.merge(customer_status, on='ID', how='left') \
    .merge(customer_balance, on='ID', how='left')
final_result['STATUS'] = final_result['STATUS'].fillna('No disponible')  # Sustituir valores nulos en Status

## Resultados

In [12]:
# Resultado final
final_result

,Correlative,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,application_date,EOMONTH_ApplicationDate,START_DATE,Status_date,STATUS,AVGDepositBalance
0,0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,...,0,0,NaN,2.0,2023-11-30,2023-08-31,2023-05-30,NaT,No disponible,NaN
1,1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,...,0,0,NaN,2.0,2023-06-06,2023-03-31,2022-12-06,2023-03-05,C,3769.74
2,2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,...,0,0,Security staff,2.0,2021-10-20,2021-07-31,2021-04-20,2021-07-04,X,1929.04
3,3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,...,1,1,Sales staff,1.0,2023-12-20,2023-09-30,2023-06-20,NaT,No disponible,NaN
4,4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,...,1,1,Sales staff,1.0,2023-09-26,2023-06-30,2023-03-26,NaT,No disponible,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438458,438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,...,0,0,NaN,1.0,2022-11-03,2022-08-31,2022-05-03,NaT,No disponible,NaN
438459,438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,...,0,0,Laborers,1.0,2023-01-24,2022-10-31,2022-07-24,NaT,No disponible,NaN
438460,438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,...,0,0,Sales staff,1.0,2022-12-22,2022-09-30,2022-06-22,NaT,No disponible,NaN
438461,438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,...,0,0,NaN,2.0,2022-03-21,2021-12-31,2021-09-21,NaT,No disponible,NaN
